In [1]:
#!/usr/bin/python3

import ROOT
ROOT.gSystem.Load("libRestFramework.so")
ROOT.gSystem.Load("libRestRaw.so")
ROOT.gSystem.Load("libRestDetector.so")

import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets

%matplotlib widget

Welcome to JupyROOT 6.26/06


In [36]:
#############################
# Set variables:
#############################

filename = "../data/R01744_00000_RawToRawSignal_XeNeIso_Calibration_57Vetoes_konrad_2.3.12_50events.root"

# noise
smoothingWindow = 75 # 75
# BaseLineRange = [5,507] # [5,55]
BaseLineRange = [10,100] # [5,55]
PointThreshold = 2.2       # 1.5   # how many sigmas above baseline to be counted as point above threshold
SignalThreshold = 1      # 1.5   # consecutive points have to have a standard deviation larger than this value time baseline fluctuation to be counted as signal
PointsOverThreshold = 2    # 4     # only data points with at least this number of consecutive points above threshold will be considered

sId = np.array([4520,4521,4523,4524,4525,4526,4527,4528,4529,4530,4531,4532,4539,4540,4541,4542,4543,4544])


#############################
# Main code
#############################
run = ROOT.TRestRun(filename)

def getSignal(i,ii):
    # Get entry
    run.GetEntry(i)

    # Get TRestRawSignalEvent
    ev = run.GetInputEvent()
    
    # translate signalID to signal index
    sIndex = ev.GetSignalIndex(int(sId[ii]))

    # Get TRestRawSignal
    signal = ev.GetSignal(sIndex)

    # use baseline correction method and fill into new signal
    signal2 = ROOT.TRestRawSignal()
    signal.GetBaseLineCorrected(signal2,smoothingWindow)
    signal2.SetID(sIndex)
        
    return signal2


def plotSignal(i,ii):

    signal = getSignal(i,ii) 
    
    # Get noise parameters
    signal.CalculateBaseLine(BaseLineRange[0],BaseLineRange[1],"")
    baseline = signal.GetBaseLine()
    baselinesigma = signal.GetBaseLineSigma()
    
    # fill signal into vector
    sSize = signal.GetNumberOfPoints()
    sVector = np.zeros(sSize)
    for i in range(sSize):
        sVector[i] = signal.GetRawData(i)

    # Use points over threshold method to separate noise from signals
    signal.InitializePointsOverThreshold(ROOT.TVector2(PointThreshold,SignalThreshold),PointsOverThreshold)
    pot1 = signal.GetPointsOverThreshold()
    sVectorPOT1 = np.zeros(len(pot1))
    for i in range(len(pot1)):
        sVectorPOT1[i] = sVector[pot1[i]]

    # Plot signal
    if (plt.fignum_exists(1)): plt.clf()
    
    plt.figure(1,figsize = (12,5))

    plt.plot(sVector) # plot raw signal
    plt.plot(pot1,sVectorPOT1,'.',color='r',label='"Points over threshold"')    # plot points over threshold
    
    # indicate region which is used for baseline determination
    plt.axvspan(BaseLineRange[0],BaseLineRange[1], color='skyblue', alpha=0.3, lw=0)

    line1 = plt.axhline(y=baseline, color='m', linestyle='-', label='standard method')
    line2 = plt.axhline(y=baseline+baselinesigma, color='m', linestyle=(0,(5,10)))
    plt.axhline(y=baseline-baselinesigma, color='m', linestyle=(0,(5,10)))
    
    plt.legend([line1,line2],["baseline, standard method","sigma, standard method"])
    
    # show if the signal is noise or not
    if len(pot1) >= PointsOverThreshold:
        plt.plot([], [], marker='o', color='w', label='Standard: SIGNAL', markerfacecolor='g', markersize=15)
    else:
        plt.plot([], [], marker='o', color='w', label='Standard: NOISE',  markerfacecolor='r', markersize=15)
    
    plt.legend()
    plt.grid(alpha=0.5)
    plt.show()

    # Print info
    print("\nbaseline = %.2f" % baseline)
    print("baseline sigma = %.2f" % baselinesigma)
    if len(pot1) >= PointsOverThreshold:
        print("SIGNAL: Points over threshold: %i" %len(pot1))
    else:
        print("NOISE: Points over threshold: %i" %len(pot1))

    

In [37]:
interact(plotSignal,i=IntSlider(description='event ID', min=0, max=50, value=0),ii=IntSlider(description='signal ID', min=0, max=17, value=1))

interactive(children=(IntSlider(value=0, description='event ID', max=50), IntSlider(value=1, description='sign…

<function __main__.plotSignal(i, ii)>